In [1]:
Project = '/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Projects/Adipocyte'
from IPython.display import display
import pandas as pd
import numpy as np
import gseapy as gp
import pickle

In [2]:
RD = pd.read_csv(f'{Project}/Results/Tables/CPM_RD_calculation/v20221217/adi_RD.tsv', sep='\t').drop(columns='gene_name')
RD.iloc[:,1:] = np.log2(RD.iloc[:,1:])

display(RD)

,gene_id,RD0a,RD0b,RD0c,RD4a,RD4b,RD4c,RD8a,RD8b,RD8c
0,ENSMUSG00000033845,0.384560,0.093573,0.176662,-0.286443,0.024544,-0.016629,0.042769,-0.347544,-0.186994
1,ENSMUSG00000025903,-4.670646,-4.631477,-4.298782,-4.865560,-4.168386,-4.135296,-5.135595,-4.469899,-4.633928
2,ENSMUSG00000033813,-1.407916,-1.127111,-1.049101,-1.416102,-0.725863,-0.987097,-1.239836,-0.961624,-0.813096
3,ENSMUSG00000033793,1.703974,1.563437,1.458909,1.767617,1.910727,1.857605,1.788015,1.926528,1.863422
4,ENSMUSG00000025907,2.355895,3.285892,2.964592,2.521603,3.628401,2.722776,2.120911,2.662165,2.845057
...,...,...,...,...,...,...,...,...,...,...
11805,ENSMUSG00000064360,-1.190822,-0.264001,1.559481,-1.171672,0.808968,-0.215973,-3.156482,-2.881273,-1.078799
11806,ENSMUSG00000064363,-3.459829,-3.415427,-3.128950,-5.300114,-3.597688,-4.175016,-5.224020,-4.328138,-4.403538
11807,ENSMUSG00000064367,-2.007124,-2.000537,-1.978983,-2.636981,-1.802296,-2.803462,-2.833428,-2.371502,-2.489688
11808,ENSMUSG00000064368,-0.916545,-1.255298,-1.037710,-1.818190,-1.401082,-1.817293,-1.631776,-1.140065,-1.393552


In [3]:
CHIP = pd.read_csv('/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Data_Library/GSEA/chip/Mouse_Ensembl_Gene_ID_MSigDB.v2023.2.Mm.chip', sep='\t', usecols=['Probe Set ID','Gene Symbol']).rename(columns={'Probe Set ID':'gene_id'})
display(CHIP)

,gene_id,Gene Symbol
0,ENSMUSG00000000001,Gnai3
1,ENSMUSG00000000003,Pbsn
2,ENSMUSG00000000028,Cdc45
3,ENSMUSG00000000031,H19
4,ENSMUSG00000000037,Scml2
...,...,...
43620,ENSMUSG00002076937,Gm25820
43621,ENSMUSG00002076971,Snord100
43622,ENSMUSG00002076975,Gm23823
43623,ENSMUSG00002076989,Gm23510


In [4]:
RD_gct = pd.merge(CHIP, RD,on='gene_id', how='right')
RD_gct = RD_gct.sort_values('gene_id').drop_duplicates(subset='Gene Symbol').set_index('Gene Symbol').drop(columns='gene_id')

display(RD_gct)

,RD0a,RD0b,RD0c,RD4a,RD4b,RD4c,RD8a,RD8b,RD8c
Gene Symbol,,,,,,,,,
Gnai3,1.273772,1.813141,1.492428,2.013561,2.055045,1.848073,1.210777,2.193114,1.875994
Cdc45,0.793407,0.980629,1.239551,0.615844,0.880220,0.821295,0.688324,1.298541,0.978301
Narf,0.100232,-0.021815,-0.137934,0.330452,0.261968,0.253551,0.624470,0.661276,0.373920
Cav2,0.532570,1.364731,0.908902,1.646404,1.970102,1.882976,1.288946,2.630021,2.026957
Klf6,0.551694,0.829447,0.671219,1.356006,1.276334,1.440677,0.745721,1.428446,1.380510
...,...,...,...,...,...,...,...,...,...
Lin54,0.522127,0.817545,0.582319,0.917551,1.342251,0.821970,0.455156,1.654602,1.714635
Ahnak2,1.078441,1.247823,1.305632,1.305619,1.541078,1.106656,0.952320,0.844563,0.925181
Rps6ka4,-0.834333,-1.016649,-0.944602,-0.582871,-1.020517,-0.829365,-0.616872,-0.983338,-1.160086


In [5]:
reps = ['a','b','c']

S2N_rnks = {}
for day in ['4','8']:
    S2N_rnks[day] = pd.DataFrame((RD_gct[[f'RD{day}{rep}' for rep in reps]].mean(axis=1) - RD_gct[[f'RD0{rep}' for rep in reps]].mean(axis=1)) / (RD_gct[[f'RD{day}{rep}' for rep in reps]].std(axis=1) + RD_gct[[f'RD0{rep}' for rep in reps]].std(axis=1))).sort_values(0, ascending=False)
display(S2N_rnks['8'])

,0
Gene Symbol,
Pkd2l2,24.399498
Cdc42bpb,13.686618
Me1,13.213630
Gtpbp8,12.017863
F2rl1,11.691288
...,...
Hcar2,-10.545614
Rbm15b,-12.124114
Cidec,-12.477998


In [6]:
def Prerank(rnk, gmt_f, threads):
    
    pre_res = gp.prerank(rnk=rnk,
                        gene_sets=gmt_f,
                        threads=threads,
                        min_size=15,
                        max_size=500,
                        permutation_num=3000, # reduce number to speed up testing
                        outdir=None, # don't write to disk
                        seed=6,
                        verbose=True, # see what's going on behind the scenes
                        )
    
    return pre_res

In [7]:
# GSEA
threads = 4
gmt_f = '/Users/daehwa/Library/CloudStorage/OneDrive-Personal/Junlab/Data_Library/GSEA/gmt/m5.go.v2023.1.Mm.symbols.gmt'

gsea_results = {}
for comp in S2N_rnks:
    rnk = S2N_rnks[comp]
    gsea_results[comp] = Prerank(rnk, gmt_f, threads)

gsea_table = {}
for comp in S2N_rnks:
    gsea_table[comp] = gsea_results[comp].res2d
    gsea_table[comp]['N'] = len(S2N_rnks[comp])

2024-05-02 14:57:15,653 [WARNING] Input gene rankings contains NA values(gene name and ranking value), drop them all!
2024-05-02 14:57:15,660 [INFO] Parsing data files for GSEA.............................
2024-05-02 14:57:15,854 [INFO] 6008 gene_sets have been filtered out when max_size=500 and min_size=15
2024-05-02 14:57:15,855 [INFO] 4665 gene_sets used for further statistical testing.....
2024-05-02 14:57:15,855 [INFO] Start to run GSEA...Might take a while..................
2024-05-02 14:59:34,942 [INFO] Congratulations. GSEApy runs successfully................

2024-05-02 14:59:34,978 [WARNING] Input gene rankings contains NA values(gene name and ranking value), drop them all!
2024-05-02 14:59:34,982 [INFO] Parsing data files for GSEA.............................
2024-05-02 14:59:35,200 [INFO] 6008 gene_sets have been filtered out when max_size=500 and min_size=15
2024-05-02 14:59:35,200 [INFO] 4665 gene_sets used for further statistical testing.....
2024-05-02 14:59:35,200 [INF

In [8]:
# Save
with open("adi_logRD_temporal_GO.pickle","wb") as fw:
    pickle.dump(gsea_table, fw)

with open("adi_logRD_temporal_GO_raw.pickle","wb") as fw:
    pickle.dump(gsea_results, fw)

In [15]:
tmp = gsea_table['4'].sort_values('FDR q-val', ascending=True)
display(tmp[:50])

,Name,Term,ES,NES,NOM p-val,FDR q-val,FWER p-val,Tag %,Gene %,Lead_genes,N
0,prerank,GOCC_INNER_MITOCHONDRIAL_MEMBRANE_PROTEIN_COMPLEX,-0.68544,-2.887136,0.0,0.0,0.0,82/141,15.28%,Atp5md;Cox8b;Atp5g1;Smdt1;Uqcr11;mt-Co3;Romo1;...,11808
11,prerank,GOBP_ATP_BIOSYNTHETIC_PROCESS,-0.61322,-2.438649,0.0,0.0,0.0,51/94,16.50%,Atp5g1;Uqcc3;Ndufs7;mt-Nd1;Ndufb7;Ndufb8;Antkm...,11808
10,prerank,GOBP_NADH_DEHYDROGENASE_COMPLEX_ASSEMBLY,-0.668345,-2.461717,0.0,0.0,0.0,37/59,16.81%,Ndufaf8;Tmem126a;Tafazzin;Ndufs7;mt-Nd1;Ndufb7...,11808
8,prerank,GOBP_MITOCHONDRIAL_RESPIRATORY_CHAIN_COMPLEX_A...,-0.63601,-2.511017,0.0,0.0,0.0,51/91,16.81%,Ndufaf8;Sdhaf1;Cox14;Tmem126a;Uqcc3;Tafazzin;N...,11808
7,prerank,GOCC_MITOCHONDRIAL_PROTEIN_CONTAINING_COMPLEX,-0.551312,-2.53272,0.0,0.0,0.0,151/285,20.14%,Mrpl54;Atp5md;Cox8b;Atp5g1;Smdt1;Mrpl18;Uqcr11...,11808
6,prerank,GOBP_AEROBIC_ELECTRON_TRANSPORT_CHAIN,-0.691284,-2.53904,0.0,0.0,0.0,34/62,13.25%,Cox8b;Uqcr11;mt-Co3;Uqcc3;Ndufs7;mt-Nd1;Cycs;U...,11808
9,prerank,GOBP_NUCLEOSIDE_TRIPHOSPHATE_BIOSYNTHETIC_PROCESS,-0.59518,-2.466272,0.0,0.0,0.0,60/115,16.50%,Atp5g1;Uck1;Uqcc3;Ndufs7;mt-Nd1;Ndufb7;Ndufb8;...,11808
4,prerank,GOCC_NADH_DEHYDROGENASE_COMPLEX,-0.76749,-2.688557,0.0,0.0,0.0,35/48,16.50%,Ndufs7;mt-Nd1;Ndufb7;Ndufb8;Ndufs6;Ndufa13;Ndu...,11808
3,prerank,GOBP_PROTON_MOTIVE_FORCE_DRIVEN_ATP_SYNTHESIS,-0.726857,-2.728964,0.0,0.0,0.0,45/67,16.50%,Atp5g1;Ndufs7;mt-Nd1;Ndufb7;Ndufb8;Antkmt;Nduf...,11808
2,prerank,GOBP_OXIDATIVE_PHOSPHORYLATION,-0.648747,-2.737832,0.0,0.0,0.0,76/135,16.50%,Cox8b;Uqcr11;mt-Co3;Uqcc3;Tafazzin;Ndufs7;mt-N...,11808


In [16]:
tmp = gsea_table['8'].sort_values('FDR q-val', ascending=True)
display(tmp[:50])

,Name,Term,ES,NES,NOM p-val,FDR q-val,FWER p-val,Tag %,Gene %,Lead_genes,N
0,prerank,GOCC_INNER_MITOCHONDRIAL_MEMBRANE_PROTEIN_COMPLEX,-0.61053,-2.651371,0.0,0.0,0.0,86/141,19.64%,Chchd3;Atp5g1;Cox8b;Uqcrb;mt-Cytb;Ndufa4;Timm1...,11808
1,prerank,GOCC_RESPIRASOME,-0.642963,-2.592375,0.0,0.0,0.0,59/89,19.74%,Cox8b;Uqcrb;mt-Cytb;Ndufa4;Ndufb7;Uqcc3;Cox7b;...,11808
2,prerank,GOCC_NADH_DEHYDROGENASE_COMPLEX,-0.701978,-2.517026,0.0,0.0,0.0,34/48,16.84%,Ndufa4;Ndufb7;Ndufb4;mt-Nd5;Ndufa13;Ndufs6;mt-...,11808
3,prerank,GOBP_PROTON_MOTIVE_FORCE_DRIVEN_ATP_SYNTHESIS,-0.624728,-2.382318,0.0,0.0,0.0,44/67,20.48%,Atp5g1;Ndufb7;Ndufb4;mt-Nd5;Ndufa13;Ndufs6;Ant...,11808
4,prerank,GOBP_OXIDATIVE_PHOSPHORYLATION,-0.532624,-2.294615,0.0,0.000737,0.003333,67/135,16.84%,Cox8b;Uqcrb;mt-Cytb;Ndufb7;Uqcc2;Uqcc3;Ndufb4;...,11808
5,prerank,GOBP_ATP_SYNTHESIS_COUPLED_ELECTRON_TRANSPORT,-0.555271,-2.186878,0.0,0.005387,0.034333,40/73,18.40%,Cox8b;Uqcrb;mt-Cytb;Uqcc3;mt-Nd5;Uqcrq;Ndufs6;...,11808
6,prerank,GOBP_AEROBIC_ELECTRON_TRANSPORT_CHAIN,-0.562681,-2.147132,0.0,0.008587,0.062333,35/62,18.40%,Cox8b;Uqcrb;mt-Cytb;Uqcc3;mt-Nd5;Uqcrq;Ndufs6;...,11808
7,prerank,GOMF_STRUCTURAL_CONSTITUENT_OF_RIBOSOME,-0.477592,-2.117041,0.0,0.01205,0.094333,67/147,24.63%,Rpl21;Rpl7a;Ubb;Mrpl33;Rpl27a;Mrpl3;Rpl10;Mrpl...,11808
8,prerank,GOBP_NEGATIVE_REGULATION_OF_LIPID_CATABOLIC_PR...,-0.697983,-2.102389,0.000912,0.014114,0.125333,7/21,5.04%,Hcar2;Cidea;Adora1;Dbi;Apoc1;Plin5;Apoa2,11808
9,prerank,GOCC_MITOCHONDRIAL_PROTEIN_CONTAINING_COMPLEX,-0.43588,-2.07318,0.0,0.018656,0.178,134/285,21.18%,Chchd3;Atp5g1;Cox8b;Uqcrb;mt-Cytb;Ndufa4;Mrpl3...,11808
